### 词向量的原理
- 文本处理的最细颗粒度为单个词（字太小了），最原始的表示是采用对所有的词，建立词库，采用onehot向量来表示每个词。

比如：有一份语料，总共包含“a”，“b”，“c”三个词，建立一个词库，那么就是一个[“a”，“b”，“c”]。当我要表示a的时候就是：[1,0,0]。这个是最原始的表达，显然当词海比较大的时候，这个向量是非常稀疏的。
- 采用labelencode方式来给每一个词，一个编号。

比如：还是上面的例子，当词库的建立方式是一个列表的时候，[“a”，“b”，“c”]，可以采用每个词对应的下标来表示该词。比如a就可以用0来表示。显然这样也是有缺点的，这些编号是无意义并且有差别的，比如苹果和香蕉还有人，分别的编码是1，1000，2。
- 词向量表达（word2vec）

通过训练语言模型，得到的中间产物来表示词。主要的模型是CBOW和Skip-gram。CBOW是建立语言模型，用某个词的上下文做输入，来预测这个词。Skip-gram与之相反。CBOW适用于小型语料库，Skip-gram适用大语料库。

#### CBOW

<img src="./img/CBOW.jpg" style="zoom:50%">

#### Skip-gram
<img src="./img/Skip-gram.jpg" style="zoom:50%">

#### 当训练好这连个语言模型之后，会得到权重矩阵$W[N,V]$，当我们设定每个词的onehot表达为$x=[1,N]$时，就可以用这个权重矩阵和onehot向量相乘，得到词向量的表达，即：
$$word2vec=X[1,N]\times W[N,V]=vec[1,V]$$
#### 下面用代码实现一下词向量的训练：

In [15]:
# 模块导入
from gensim.models import word2vec

In [16]:
# 定义一个句子生成器
class MySentences(object):
    def __init__(self, fname):
        self.fname = fname    
    def __iter__(self):
        for line in open(self.fname,'r'):            
            yield line.split()

In [17]:
# 训练
DataDir = "./"
ModelDir = "./"
#词频低于该值的舍去
MIN_COUNT = 4
# 需要预先安装 Cython 以支持并行
CPU_NUM = 2 
# 设置词向量维度
VEC_SIZE = 20
# 提取目标词上下文距离最长5个词
CONTEXT_WINDOW = 5 

In [18]:
def w2vTrain(f_input, model_output):         

    sentences = MySentences(DataDir+f_input)
    # 这里是一个循环训练过程，做了２个操作，第一建立词库，第二用神经网络训练
    # 也可以分开
    #　model = gensim.models.Word2Vec(iter=1)  # an empty model, no training yet
    #　model.build_vocab(some_sentences)  # 建立此表
    #　model.train(other_sentences)  # 训练
    w2v_model = word2vec.Word2Vec(sentences, 
                                  sg=0,# cbow,1=skip-gram
                                  min_count = MIN_COUNT, 
                                  workers = CPU_NUM, 
                                  size = VEC_SIZE,
                                  window = CONTEXT_WINDOW)
    # 保存模型
    w2v_model.save(ModelDir+model_output)

In [19]:
f_input = "bioCorpus_5000.txt"
model_output = "test_w2v_model"
w2vTrain(f_input, model_output)

In [39]:
# 训练好的词向量模型才有效,初始的词向量是用初始化的参数矩阵算出来的.
# 加载训练好的模型
selfmodel = word2vec.Word2Vec.load(ModelDir+model_output)
# 输出某个词的向量
selfmodel.wv['i']

array([-0.01855507, -0.01266545,  0.00810266, -0.01430729,  0.01543698,
       -0.01801598,  0.01267219,  0.00947818,  0.02281465, -0.01764751,
        0.00306669,  0.00366936, -0.01334176, -0.00713251,  0.01816056,
       -0.01962536,  0.01906137, -0.01963314, -0.01498264,  0.00227621],
      dtype=float32)

In [ ]:
selfmodel.most_similar(positive=['woman', 'king'], negative=['man'])
selfmodel.doesnt_match("breakfast cereal dinner lunch".split())
selfmodel.similarity('woman', 'man')
#输出0.73723527 
selfmodel.wv['computer'] 